## Convergence Benchmark

In [ ]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from qiskit.quantum_info import random_unitary
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from experiments import *
#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=3)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [54]:
def generate_spam_benchmark(n=3, c1=1, c2=1):
    d = 2**n
    spam_target = SPAM(d=d)

    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    return spam_target


def generate_spam_data(spam_target, N_spam=None, noise=0):
    n = spam_target.n
    inputs_spam, _ = generate_pauliInput_circuits(n, N_spam)
    N_spam = inputs_spam.shape[0]

    state = tf.repeat(spam_target.init[None,:,:], N_spam, axis=0)
    state = apply_unitary(state, inputs_spam)
    targets_spam = measurement(state, povm = spam_target.povm)

    #add noise
    targets_spam = add_noise_to_probs(targets_spam, noise=noise)
    return inputs_spam, targets_spam


def generate_map_data(channel_target, spam_target, N_map=None, noise=0):
    n = int(np.log2(channel_target.d))
    inputs_map, _ = generate_pauli_circuits(n = n, 
                                            circuit_target=None,  
                                            trace=False,
                                            N=N_map)
    U_prep, U_basis = inputs_map

    N_map = U_prep.shape[0]
    state = tf.repeat(tf.expand_dims(spam_target.init, axis=0), N_map, axis=0)
    state = apply_unitary(state, U_prep)
    state = channel_target.apply_channel(state)
    targets_map = measurement(state, U_basis, spam_target.povm)
    
    return inputs_map, targets_map


def povm_fidelity(spam_a, spam_b):
    d = spam_a.d
    povm_a, povm_b = spam_a.povm, spam_b.povm
    ab = tf.matmul(povm_a, povm_b)
    ab_sqrt = tf.linalg.sqrtm(ab)
    fidelity = tf.math.reduce_sum(tf.linalg.trace(ab_sqrt))/d
    return fidelity


def model_pipeline(channel_target, noise, N_map=2000, rank = None):
    # Make Benchmark
    #################################################################################
    d = channel_target.d
    if rank is None:
        rank = d**2
    
    spam_target = generate_spam_benchmark(n=n, c1=0.8, c2=0.8)
    inputs_spam, targets_spam = generate_spam_data(spam_target, N_spam=None, noise=noise)

    inputs_map, targets_map = generate_map_data(channel_target, spam_target, N_map=N_map, noise=noise)
    #################################################################################

    # Fit Models
    #################################################################################
    spam_model = SPAM(d=d,
                      use_corr_mat=True,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))

    spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                        num_iter = 300,
                        verbose = False,
                        )

    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 2000,
                     verbose = False,
                    )

    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                               rank = rank,
                                               spam = spam_model,
                                              ),
                            loss_function = ProbabilityMSE(),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            logger = Logger(loss_function = channel_fidelity_loss),
                           )

    model.train(inputs = inputs_map,
                targets = targets_map,
                inputs_val = None,
                targets_val = [channel_target],
                num_iter = 4000,
                N = 500,
                )
    #################################################################################

    return model

## Retrieve SPAM and Map

In [61]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 1
d = 2**n

channel_target = DilutedKrausMap( 
                               U=generate_unitary(d=d), 
                               c=0.5, 
                               d=d, 
                               rank=d-1,
                              )
model_list1 = []
for i in tqdm(range(10)):
    channel_target = DilutedKrausMap( 
                               U=generate_unitary(d=d), 
                               c=0.5, 
                               d=d, 
                               rank=d-1,
                              )
    
    model = model_pipeline(channel_target, noise = 1/np.sqrt(1000), N_map=None)
    model_list1.append(model)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.00036975542814663326


  0%|          | 0/4000 [00:00<?, ?it/s]

None -0.47840024968070693
None -0.8818654662632592
None -0.9445144916046343
None -0.9645375887617914
None -0.9736241060739039
None -0.9784594646977062
None -0.9812544853433097
None -0.9829409947260024
None -0.9839801065229291
None -0.9846261451917957
None -0.9850286612673446
None -0.9852782475296944
None -0.985430913807518
None -0.985522105847466
None -0.9855748094555693
None -0.9856040310404746
None -0.9856195050741451
None -0.9856272950279134
None -0.9856309968278233
None -0.985632683651716
None -0.9856333929610541
None -0.9856336709341614
None -0.9856337699817772
None -0.9856338204539934
None -0.9856338239495299
None -0.9856338345483462
None -0.9856338258842039
None -0.9856338330340106
None -0.9856338273781254
None -0.9856338356946689
None -0.9856338339774281
None -0.9856338212084098
None -0.9856338244572193
None -0.9856338261939159
None -0.9856338195360916
None -0.985633818505134
None -0.9856338256048649
None -0.9856338269710687
None -0.9856338216075744
None -0.9856338221172848
Non

...

None -0.43048324934255017
None -0.8810728618701046
None -0.9482807535463295
None -0.964843498883165
None -0.9710907436903077
None -0.9737798636185521
None -0.9750483290945287
None -0.9757024393662319
None -0.9760463291651513
None -0.9762163024086182
None -0.9762922708754609
None -0.9763226998209605
None -0.9763335926875956
None -0.97633709913257
None -0.9763381059620374
None -0.9763383535951872
None -0.9763384113399273
None -0.9763384262863704
None -0.9763384384343983
None -0.9763384278535088
None -0.9763384380804896
None -0.976338425143537
None -0.9763384307929152
None -0.9763384308292811
None -0.9763384293688361
None -0.9763384228249777
None -0.9763384278240338
None -0.9763384292047015
None -0.9763384345569187
None -0.9763384276514007
None -0.9763384252705882
None -0.9763384284351471
None -0.9763384214749109
None -0.9763384280336367
None -0.97633843100306
None -0.9763384306091498
None -0.9763384237211598
None -0.9763384305530927
None -0.9763384224086658
None -0.9763384260619669
None 

  0%|          | 0/2000 [00:00<?, ?it/s]

0.0002661871795034816


  0%|          | 0/4000 [00:00<?, ?it/s]

None -0.697449673800992
None -0.9588519255064674
None -0.9823180345492427
None -0.9874380906725633
None -0.9888329309161494
None -0.9892444154793708
None -0.9893592106916771
None -0.989387277172133
None -0.9893931754786551
None -0.9893942306388223
None -0.989394382373399
None -0.9893944107359487
None -0.9893944120960552
None -0.9893944004012942
None -0.9893944055914674
None -0.9893944104203348
None -0.9893944037649534
None -0.9893944158635991
None -0.9893944022303518
None -0.9893944003224621
None -0.989394404871371
None -0.9893944051001302
None -0.9893944074901643
None -0.9893944081036338
None -0.9893944118210514
None -0.9893944000406562
None -0.9893944090683766
None -0.9893944064102956
None -0.9893944087145363
None -0.9893944128414575
None -0.9893944080964988
None -0.9893944036425035
None -0.9893944138593365
None -0.9893944114274801
None -0.9893943998218864
None -0.9893944060371728
None -0.9893944095330391
None -0.9893944021038805
None -0.9893944194605824
None -0.9893944204798695
None

In [62]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 2
d = 2**n

channel_target = DilutedKrausMap( 
                               U=generate_unitary(d=d), 
                               c=0.5, 
                               d=d, 
                               rank=d-1,
                              )
model_list2 = []
for i in tqdm(range(10)):
    channel_target = DilutedKrausMap( 
                               U=generate_unitary(d=d), 
                               c=0.5, 
                               d=d, 
                               rank=d-1,
                              )
    
    model = model_pipeline(channel_target, noise = 1/np.sqrt(1000), N_map=2000)
    model_list2.append(model)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.004056401117446949


  0%|          | 0/4000 [00:00<?, ?it/s]

None -0.18367121661425423
None -0.6468194163295719
None -0.8359949516962233
None -0.8994743525119361
None -0.9271943519754156
None -0.9408380802823219
None -0.9479949566022816
None -0.9519628531812101
None -0.9542841882580451
None -0.9557099631159944
None -0.9566252128178502
None -0.9572383585279818
None -0.9576665730350928
None -0.9579771768520816
None -0.9582095503755047
None -0.9583874707121852
None -0.9585259048226076
None -0.9586348136660312
None -0.9587211458853893
None -0.9587899306868922
None -0.9588449238962997
None -0.9588889811011101
None -0.958924314782693
None -0.9589526430455986
None -0.9589753293701877
None -0.9589934595556416
None -0.9590079176098704
None -0.959019391339723
None -0.9590284772636453
None -0.9590356441517714
None -0.9590412702228893
None -0.9590456638015942
None -0.9590490770693597
None -0.9590517306257755
None -0.9590537821306142
None -0.9590553629220687
None -0.9590565758056386
None -0.9590575079996735
None -0.9590582234548835
None -0.9590587563288948
N

...

None -0.2047486281389309
None -0.678871376803652
None -0.8452001426492889
None -0.9027803812943604
None -0.9280917206053179
None -0.940750920106915
None -0.9476647396054306
None -0.95171452176802
None -0.9542339735381994
None -0.9558830279951738
None -0.9570071834282217
None -0.9577980074099782
None -0.958367677001899
None -0.9587854018196462
None -0.9590958546897742
None -0.9593289928028376
None -0.9595055740936077
None -0.9596402878551219
None -0.9597436898176405
None -0.9598234822468481
None -0.9598853144807661
None -0.9599334259146858
None -0.9599709407454715
None -0.9600002712406493
None -0.9600232028394099
None -0.9600411409638994
None -0.9600551457563461
None -0.9600660859430801
None -0.9600745961359445
None -0.9600812017417731
None -0.9600863133420189
None -0.9600902439149368
None -0.9600932595392266
None -0.9600955777244712
None -0.9600973454080607
None -0.9600986765052205
None -0.9600996881524707
None -0.9601004472745545
None -0.9601010243766214
None -0.9601014534581269
None 

  0%|          | 0/2000 [00:00<?, ?it/s]

0.0032645330878243624


  0%|          | 0/4000 [00:00<?, ?it/s]

None -0.15642097004363478
None -0.6210478947734223
None -0.8381295741240148
None -0.9050584069542064
None -0.9318241041147115
None -0.9443017191444364
None -0.9507706840388578
None -0.9544338725293304
None -0.9566715070737106
None -0.9581283427049585
None -0.9591257314274801
None -0.9598340835215488
None -0.9603502077645245
None -0.9607331801925042
None -0.961021290781274
None -0.9612405197419182
None -0.9614090029583906
None -0.9615396264153249
None -0.9616417493579624
None -0.9617221769419109
None -0.9617859559754443
None -0.961836840970262
None -0.9618776536255975
None -0.9619105475560744
None -0.9619371518502864
None -0.9619587453525382
None -0.9619762877236888
None -0.9619905796350864
None -0.9620022097543643
None -0.9620116688976904
None -0.9620193619162178
None -0.9620256069258885
None -0.9620306579715806
None -0.9620347300936417
None -0.9620379879542182
None -0.9620405876720101
None -0.9620426501416832
None -0.9620442708756777
None -0.962045540011195
None -0.9620465209048832
No

In [ ]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n

channel_target = DilutedKrausMap( 
                               U=generate_unitary(d=d), 
                               c=0.5, 
                               d=d, 
                               rank=d-1,
                              )
model_list3 = []
for i in tqdm(range(10)):
    channel_target = DilutedKrausMap( 
                               U=generate_unitary(d=d), 
                               c=0.5, 
                               d=d, 
                               rank=d-1,
                              )
    
    model = model_pipeline(channel_target, noise = 1/np.sqrt(1000), N_map=5000)
    model_list3.append(model)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.006681537840839778


  0%|          | 0/4000 [00:00<?, ?it/s]

None -0.10550747757372507
None -0.4258479205200387
None -0.6363082965581386
None -0.7449584162157769
None -0.8116572939831738
None -0.8537869254773415
None -0.8812479661071544
None -0.8994322758063212
None -0.9117625952506097
None -0.9204364592092629
None -0.9264663193049683
None -0.930838701088791
None -0.934065428494107
None -0.9364048523690761
None -0.9383445874805946
None -0.9395489515642418
None -0.9406835323634151
None -0.9414158941015924
None -0.9419964276999303
None -0.942886928143986
None -0.9432568692785255
None -0.9437121029654558
None -0.9437913729005677
None -0.9441932644210745
None -0.9446085721008559
None -0.9446010945607335
None -0.9447079794161979
None -0.9451158818312411
None -0.9451896375946862
None -0.9450626032639032
None -0.9451664371015686
None -0.9455018101883154
None -0.9454945506073892
None -0.9455055337134793
None -0.9456025440467453
None -0.945780483002317
None -0.9457230145070247
None -0.945700968659485
None -0.9460554574337015
None -0.9459963463708495
None

...

None -0.10296835943716806
None -0.422618637515805
None -0.6228064463516042
None -0.7304127040911276
None -0.799835440980499
None -0.8447885799402413
None -0.8744703562298437
None -0.8946784546956675
None -0.9084057285437883
None -0.9180187105535159
None -0.9247865562022181
None -0.9300029221153733
None -0.9336256695432501
None -0.9361777416531625
None -0.9380263198614118
None -0.9395505235191369
None -0.9407910265411679
None -0.9418433398899325
None -0.9428537171119798
None -0.9433505270502861
None -0.943938282501587
None -0.9443594199150548
None -0.9446801721339335
None -0.9449743523092798
None -0.9451665316578857
None -0.9453565972062964
None -0.9455337886548566
None -0.9458332022196408
None -0.9460032866235989
None -0.945981408844147
None -0.9462627901759917
None -0.9464550648245958
None -0.9464271164071034
None -0.9465039337932811
None -0.9463940400671541
None -0.9462400740003717
None -0.9465748548124729
None -0.9465630100319987
None -0.9465410283062355
None -0.9464382887036532
Non

  0%|          | 0/2000 [00:00<?, ?it/s]

0.006873328096967519


  0%|          | 0/4000 [00:00<?, ?it/s]

None -0.09075216044067229
None -0.4058750989791703
None -0.6133647583144438
None -0.7253906918804469
None -0.7962232566726598
None -0.8422373286913006
None -0.8733339843077235
None -0.894502561101734
None -0.9094477489822903
None -0.9201854953547024
None -0.9279891139393623
None -0.9336863374383991
None -0.9380345431670037
None -0.941213628574153
None -0.9435969313300935
None -0.9457576019681163
None -0.947096670419115
None -0.9483064795370874
None -0.9491170952130389
None -0.9497548762636783
None -0.9505150717493805
None -0.9510612672847097
None -0.9517556405537216
None -0.9518322087813413
None -0.9522569601876815
None -0.9524621347230987
None -0.9527401978935248
None -0.9528145120404513
None -0.9532298168688473
None -0.9534121941341389
None -0.9534878233803353
None -0.9535022872876626
None -0.9537051020800181
None -0.9534638795858232
None -0.9535953763760128
None -0.9537611471647051
None -0.9539080668388406
None -0.9537639103810898
None -0.9537835115722415
None -0.954164100265328
Non

In [ ]:
fidelity_twoQubits = [model.logger.loss_val_list for model in model_list1]
fidelity_threeQubits = [model.logger.loss_val_list for model in model_list2]
fidelity_fourQubits = [model.logger.loss_val_list for model in model_list3]

In [ ]:
fig = plt.figure(figsize=(6, 4), dpi=300, facecolor="w", edgecolor="k")

plt.plot(-np.array(fidelity_twoQubits).T,"g-")
plt.plot(-np.array(fidelity_threeQubits).T,"b-")
plt.plot(-np.array(fidelity_fourQubits).T,"r-")
#plt.yscale("log")
plt.ylim([0.9, 1.05])
plt.grid()
plt.show()